Dataset from https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset?resource=download
https://towardsdatascience.com/bert-text-classification-using-pytorch-723dfb8b6b5b used this and simply converted to spam classifier

In [2]:
import pandas as pd

datapath = "spam.csv"
df = pd.read_csv(datapath, encoding='latin-1')
df.head()


,category,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [10]:
import torch
import numpy as np
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
labels = {'ham':0,
          'spam':1,
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['category']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 512, truncation=True,
                                return_tensors="pt") for text in df['text']]

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [11]:
np.random.seed(112)
df_train, df_val, df_test = np.split(df.sample(frac=1, random_state=42), 
                                     [int(.8*len(df)), int(.9*len(df))])

print(len(df_train),len(df_val), len(df_test))

4457 557 558


In [12]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [15]:
from torch.optim import Adam
from tqdm import tqdm

def train(model, train_data, val_data, learning_rate, epochs):

    train, val = Dataset(train_data), Dataset(val_data)

    train_dataloader = torch.utils.data.DataLoader(train, batch_size=2, shuffle=True)
    val_dataloader = torch.utils.data.DataLoader(val, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr= learning_rate)

    if use_cuda:
            print("Using gpu")
            model = model.cuda()
            criterion = criterion.cuda()

    for epoch_num in range(epochs):

            total_acc_train = 0
            total_loss_train = 0

            for train_input, train_label in tqdm(train_dataloader):

                train_label = train_label.to(device)
                mask = train_input['attention_mask'].to(device)
                input_id = train_input['input_ids'].squeeze(1).to(device)

                output = model(input_id, mask)
                
                batch_loss = criterion(output, train_label.long())
                total_loss_train += batch_loss.item()
                
                acc = (output.argmax(dim=1) == train_label).sum().item()
                total_acc_train += acc

                model.zero_grad()
                batch_loss.backward()
                optimizer.step()
            
            total_acc_val = 0
            total_loss_val = 0

            with torch.no_grad():

                for val_input, val_label in val_dataloader:

                    val_label = val_label.to(device)
                    mask = val_input['attention_mask'].to(device)
                    input_id = val_input['input_ids'].squeeze(1).to(device)

                    output = model(input_id, mask)

                    batch_loss = criterion(output, val_label.long())
                    total_loss_val += batch_loss.item()
                    
                    acc = (output.argmax(dim=1) == val_label).sum().item()
                    total_acc_val += acc
            
            print(
                f'Epochs: {epoch_num + 1} | Train Loss: {total_loss_train / len(train_data): .3f} \
                | Train Accuracy: {total_acc_train / len(train_data): .3f} \
                | Val Loss: {total_loss_val / len(val_data): .3f} \
                | Val Accuracy: {total_acc_val / len(val_data): .3f}')
                  
EPOCHS = 5
model = BertClassifier()
LR = 1e-6
              
train(model, df_train, df_val, LR, EPOCHS)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2229/2229 [1:09:11<00:00,  1.86s/it]


Epochs: 1 | Train Loss:  0.196                 | Train Accuracy:  0.890                 | Val Loss:  0.031                 | Val Accuracy:  0.991


100%|██████████| 2229/2229 [1:12:51<00:00,  1.96s/it]


Epochs: 2 | Train Loss:  0.021                 | Train Accuracy:  0.994                 | Val Loss:  0.016                 | Val Accuracy:  0.995


100%|██████████| 2229/2229 [1:09:24<00:00,  1.87s/it]


Epochs: 3 | Train Loss:  0.010                 | Train Accuracy:  0.997                 | Val Loss:  0.009                 | Val Accuracy:  0.995


100%|██████████| 2229/2229 [1:10:31<00:00,  1.90s/it]


Epochs: 4 | Train Loss:  0.004                 | Train Accuracy:  0.999                 | Val Loss:  0.012                 | Val Accuracy:  0.993


100%|██████████| 2229/2229 [1:09:35<00:00,  1.87s/it]


Epochs: 5 | Train Loss:  0.002                 | Train Accuracy:  0.999                 | Val Loss:  0.011                 | Val Accuracy:  0.993


In [30]:
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=2)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()

    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_label in test_dataloader:

              test_label = test_label.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)

              acc = (output.argmax(dim=1) == test_label).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    
evaluate(model, df_test)

Test Accuracy:  0.980


In [33]:
class Predict:
        def __init__(self, model, tokenizer):
    
            self.model = model
            self.tokenizer = tokenizer
    
        def predict(self, text):
            device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
            text = self.tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
            mask = text['attention_mask'].to(device)
            input_id = text['input_ids'].squeeze(1).to(device)
    
            output = self.model(input_id, mask)
    
            return output.argmax(dim=1).item()

In [38]:
p = Predict(model, tokenizer)
out0 = p.predict("BENDIGO: Your services are on hold, Please confirm some activity via http://Secure-onliine-bankingau.com")
out1 = p.predict("Hi, was wondering if you could make the meeting today to discuss the bank transfer?")
out2 = p.predict("Congratulations! You have won a free holiday to the Bahamas! Please call 0800 123 4567 to claim your prize.")
print(out0)
print(out1)
print(out2)

# 0 = ham, 1 = spam

1
0
1
